In [1]:
import datetime
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px # to plot the time series plot
from sklearn import metrics # for the evaluation
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
import tensorflow as tf 
from scipy import stats

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
import itertools

import warnings
warnings.filterwarnings("ignore")

# I. Load data

In [35]:
OF = pd.read_pickle("Related_dataset/Opinion_Finder_result.pkl")
GPOMS = pd.read_pickle("Related_dataset/GPOMS_result.pkl")
DJIA = pd.read_pickle("Related_dataset/DJIA_filled_missing_value.pkl")

# 2. Prepare dataset

In [36]:
# process GPOMS dataframe
GPOMS_pivot = GPOMS.drop(columns=['tweets_processed', 'tweet_size', 'GPOMS_preprocess_data', 'GPOMS_sum','sum_index', 'avg_index'])
GPOMS_pivot[['Calm','Kind','Happy','Alert','Sure','Vital']] = pd.DataFrame(GPOMS.GPOMS_avg.tolist(), index= GPOMS.index)
GPOMS_pivot = GPOMS_pivot.drop(['GPOMS_avg'], axis=1)
GPOMS_pivot = GPOMS_pivot.reset_index()
GPOMS_pivot = GPOMS_pivot.rename({'date_prune': 'Dates'}, axis=1) 

# process DJIA dataframe
DJIA['Dates'] = DJIA['Dates'].astype(str)

combine_df = DJIA.merge(GPOMS_pivot, on='Dates')
combine_df

,Dates,Close,Calm,Kind,Happy,Alert,Sure,Vital
0,2020-04-09,23719.369141,0.073364,0.191523,-0.115305,0.346767,0.136158,0.155670
1,2020-04-10,23555.069336,0.092862,0.195467,-0.102743,0.342314,0.130042,0.118410
2,2020-04-11,23472.919434,0.061887,0.278887,-0.064833,0.340592,0.101464,0.160457
3,2020-04-12,23431.844482,0.082763,0.216780,-0.062001,0.329557,0.111304,0.079260
4,2020-04-13,23390.769531,0.079693,0.222743,-0.091400,0.338160,0.130949,0.137338
...,...,...,...,...,...,...,...,...
72,2020-07-12,26083.175781,0.078391,0.181213,-0.097056,0.369545,0.147573,0.078480
73,2020-07-13,26085.800781,0.082311,0.183053,-0.082040,0.357759,0.154204,0.122646
74,2020-07-14,26642.589844,0.098220,0.186504,-0.089044,0.359141,0.150973,0.111742
75,2020-07-15,26870.099609,0.084568,0.184610,-0.079527,0.337375,0.146182,0.102206


# III. Build Neural Network model to predict DJIA
input: 
1. The past 5 days of DJIA values
2. Opinion Finder mood data (past 5 days)

Time period (in thesis): 
> Train: 2008-2-28 ~ 2008-11-28 (10 months)

> Test: 2008-12-1 ~ 2008-12-19 (19 days)

Time period (in this project): 
> Train: 2020-4-9 ~ 2020-6-23 (54 days, some missing data)

> Test: 2020-6-24 ~ 2022-7-16 (18 days)


In [37]:
threshold = 54
WINDOW_SIZE = 5

### Part 2: 提供過去3天的DJIA + GPOMS 資料: 

multi-variable

> X[0]: [[ [1-1, 1-2, ..], [2-1, 2-2, ...], [3-1, 3-2, ...]]]

> y[0]: [4]


In [38]:
temp2 = combine_df.set_index('Dates')
mms_close = MinMaxScaler()
mms_calm = MinMaxScaler()
mms_kind = MinMaxScaler()
mms_happy = MinMaxScaler()
mms_alert = MinMaxScaler()
mms_sure = MinMaxScaler()
mms_vital = MinMaxScaler()
mms_delta = MinMaxScaler()

temp2['Close'] = mms_close.fit_transform(np.array(temp2['Close']).reshape(-1, 1))
temp2['Calm'] = mms_calm.fit_transform(np.array(temp2['Calm']).reshape(-1, 1))
temp2['Kind'] = mms_kind.fit_transform(np.array(temp2['Kind']).reshape(-1, 1))
temp2['Happy'] = mms_happy.fit_transform(np.array(temp2['Happy']).reshape(-1, 1))
temp2['Alert'] = mms_alert.fit_transform(np.array(temp2['Alert']).reshape(-1, 1))
temp2['Sure'] = mms_sure.fit_transform(np.array(temp2['Sure']).reshape(-1, 1))
temp2['Vital'] = mms_vital.fit_transform(np.array(temp2['Vital']).reshape(-1, 1))

temp2.head()

,Close,Calm,Kind,Happy,Alert,Sure,Vital
Dates,,,,,,,
2020-04-09,0.153833,0.710392,0.416849,0.471563,0.458773,0.248355,0.778378
2020-04-10,0.117752,0.838021,0.443179,0.596098,0.437843,0.204577,0.468202
2020-04-11,0.099711,0.635268,1.000000,0.971922,0.429748,0.000000,0.818231
2020-04-12,0.090690,0.771914,0.585441,1.000000,0.377869,0.070439,0.142280
2020-04-13,0.081670,0.751823,0.625244,0.708548,0.418315,0.211071,0.625771


In [31]:
def df_to_X_y2(df, window_size=3): # transfrom input data
    df_as_np = df.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np)-window_size):
        row = [r for r in df_as_np[i:i+window_size]]
        X.append(row)
        label = df_as_np[i+window_size][0]
        y.append(label)
    return np.array(X), np.array(y)

In [32]:
def plot_modal_res(df, tp): # plot the train/test data result
    plt.title(f"{tp} result")
    plt.plot(df[f'{tp} Predictions'], c='b')
    plt.plot(df['Actuals'], c='g')
    plt.legend(["Modal Predict", "Ground True"], loc ="lower right")
    plt.show()

In [8]:
# Calculate the Mape and Directional correction rate
def mape(actual, predict): 
    actual, predict = np.array(actual), np.array(predict)
    return np.mean(np.abs((actual - predict) / actual)) * 100

def direction(actual, predict): 
    prev_act = actual[0]
    prev_pred = predict[0]
    true = []
    res = []
    for i, j in zip(actual[1:], predict[1:]): 
        if i>=prev_act:
            true.append(1)
        else:
            true.append(-1)
        if j>=prev_pred:
            res.append(1)
        else:
            res.append(-1)
        prev_act = i
        prev_pred = j
    denorminator = len(actual) -1
    numerator = 0
    for i, j in zip(true, res):
        if i==j:
            numerator += 1
    return (numerator / denorminator)

***Start iteration***

In [39]:
type = combine_df.columns[2:]
type

Index(['Calm', 'Kind', 'Happy', 'Alert', 'Sure', 'Vital'], dtype='object')

In [11]:
def eval(results):
    MAPE = mape(results['Actuals'], results['Test Predictions']).round(3)
    Direction =  direction(results['Actuals'], results['Test Predictions'])
    return MAPE, Direction

In [42]:
finish = []

num = [2, 3, 4, 5, 6, 7]
for j in num:
    NUM_FEATURE = j
    sum = list(itertools.combinations(type, NUM_FEATURE-1))

    for i in sum:
        k = ['Close']+list(i)
        tmp = temp2[k]
        
        X2, y2 = df_to_X_y2(tmp, WINDOW_SIZE)
        X2.shape, y2.shape # number of data, # size of window, number of features

        X_train2, y_train2 = X2[:threshold-3], y2[:threshold-3]
        X_test2, y_test2 = X2[threshold:], y2[threshold:]

        model2 = Sequential()
        model2.add(InputLayer((WINDOW_SIZE, NUM_FEATURE)))
        model2.add(LSTM(64))
        model2.add(Dense(8, 'relu'))
        model2.add(Dense(1, 'linear'))
        model2.summary()

        model2.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.01), metrics=[RootMeanSquaredError()])
        model2.fit(X_train2, y_train2, epochs=10)

        # train result
        train_predictions2 = model2.predict(X_train2).flatten()
        pred = mms_close.inverse_transform(train_predictions2.reshape(-1, 1))
        pred = pred.reshape(-1,)
        act = mms_close.inverse_transform(y_train2.reshape(-1, 1))
        act = act.reshape(-1,)

        train_results2 = pd.DataFrame(data={
            'Train Predictions':pred, 
            'Actuals':act})

        # Test result
        test_predictions2 = model2.predict(X_test2).flatten()
        pred = mms_close.inverse_transform(test_predictions2.reshape(-1, 1))
        pred = pred.reshape(-1,)
        act = mms_close.inverse_transform(y_test2.reshape(-1, 1))
        act = act.reshape(-1,)

        test_results2 = pd.DataFrame(data={
            'Test Predictions':pred, 
            'Actuals':act})
        
        e1, e2 = eval(test_results2)
        finish.append([i, e1, e2])
        del model2

Model: "sequential_63"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_63 (LSTM)              (None, 64)                17152     
                                                                 
 dense_126 (Dense)           (None, 8)                 520       
                                                                 
 dense_127 (Dense)           (None, 1)                 9         
                                                                 
Total params: 17,681
Trainable params: 17,681
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
2/2 [==============================] - 3s 6ms/step - loss: 0.2057 - root_mean_squared_error: 0.4536
Epoch 2/10
2/2 [==============================] - 0s 7ms/step - loss: 0.0757 - root_mean_squared_error: 0.2752
Epoch 3/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0437 - root_mean_sq

 Layer (type)                Output Shape              Param #   
 lstm_67 (LSTM)              (None, 64)                17152     
                                                                 
 dense_134 (Dense)           (None, 8)                 520       
                                                                 
 dense_135 (Dense)           (None, 1)                 9         
                                                                 
Total params: 17,681
Trainable params: 17,681
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
2/2 [==============================] - 3s 7ms/step - loss: 0.3210 - root_mean_squared_error: 0.5666
Epoch 2/10
2/2 [==============================] - 0s 6ms/step - loss: 0.1177 - root_mean_squared_error: 0.3431
Epoch 3/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0455 - root_mean_squared_error: 0.2132
Epoch 4/10
2/2 [==============================] - 0s 5ms/step - loss:

 lstm_71 (LSTM)              (None, 64)                17408     
                                                                 
 dense_142 (Dense)           (None, 8)                 520       
                                                                 
 dense_143 (Dense)           (None, 1)                 9         
                                                                 
Total params: 17,937
Trainable params: 17,937
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
2/2 [==============================] - 2s 6ms/step - loss: 0.1730 - root_mean_squared_error: 0.4159
Epoch 2/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0957 - root_mean_squared_error: 0.3094
Epoch 3/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0448 - root_mean_squared_error: 0.2116
Epoch 4/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0504 - root_mean_squared_error: 0.2244
Epoch 5/10
2/2 [========

 lstm_75 (LSTM)              (None, 64)                17408     
                                                                 
 dense_150 (Dense)           (None, 8)                 520       
                                                                 
 dense_151 (Dense)           (None, 1)                 9         
                                                                 
Total params: 17,937
Trainable params: 17,937
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
2/2 [==============================] - 3s 5ms/step - loss: 0.1155 - root_mean_squared_error: 0.3399
Epoch 2/10
2/2 [==============================] - 0s 6ms/step - loss: 0.1699 - root_mean_squared_error: 0.4122
Epoch 3/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0423 - root_mean_squared_error: 0.2057
Epoch 4/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0892 - root_mean_squared_error: 0.2987
Epoch 5/10
2/2 [========

                                                                 
 dense_158 (Dense)           (None, 8)                 520       
                                                                 
 dense_159 (Dense)           (None, 1)                 9         
                                                                 
Total params: 17,937
Trainable params: 17,937
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
2/2 [==============================] - 3s 5ms/step - loss: 0.3847 - root_mean_squared_error: 0.6203
Epoch 2/10
2/2 [==============================] - 0s 6ms/step - loss: 0.2525 - root_mean_squared_error: 0.5025
Epoch 3/10
2/2 [==============================] - 0s 5ms/step - loss: 0.2150 - root_mean_squared_error: 0.4637
Epoch 4/10
2/2 [==============================] - 0s 5ms/step - loss: 0.1640 - root_mean_squared_error: 0.4050
Epoch 5/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0747 - root_mean_s

...

_________________________________________________________________
Epoch 1/10
2/2 [==============================] - 2s 11ms/step - loss: 0.2014 - root_mean_squared_error: 0.4488
Epoch 2/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0330 - root_mean_squared_error: 0.1817
Epoch 3/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0581 - root_mean_squared_error: 0.2410
Epoch 4/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0223 - root_mean_squared_error: 0.1492
Epoch 5/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0240 - root_mean_squared_error: 0.1550
Epoch 6/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0332 - root_mean_squared_error: 0.1823
Epoch 7/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0285 - root_mean_squared_error: 0.1690
Epoch 8/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0216 - root_mean_squared_error: 0.1470
Epoch 9/10
2/2 [=============================

Epoch 1/10
2/2 [==============================] - 2s 8ms/step - loss: 0.2953 - root_mean_squared_error: 0.5434
Epoch 2/10
2/2 [==============================] - 0s 6ms/step - loss: 0.1344 - root_mean_squared_error: 0.3665
Epoch 3/10
2/2 [==============================] - 0s 6ms/step - loss: 0.1075 - root_mean_squared_error: 0.3278
Epoch 4/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0612 - root_mean_squared_error: 0.2473
Epoch 5/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0678 - root_mean_squared_error: 0.2603
Epoch 6/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0614 - root_mean_squared_error: 0.2478
Epoch 7/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0418 - root_mean_squared_error: 0.2044
Epoch 8/10
2/2 [==============================] - 0s 8ms/step - loss: 0.0421 - root_mean_squared_error: 0.2051
Epoch 9/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0352 - root_mean_squared_error: 0.1875
E

2/2 [==============================] - 3s 11ms/step - loss: 0.2412 - root_mean_squared_error: 0.4911
Epoch 2/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0803 - root_mean_squared_error: 0.2835
Epoch 3/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0669 - root_mean_squared_error: 0.2587
Epoch 4/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0501 - root_mean_squared_error: 0.2237
Epoch 5/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0401 - root_mean_squared_error: 0.2003
Epoch 6/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0326 - root_mean_squared_error: 0.1805
Epoch 7/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0370 - root_mean_squared_error: 0.1923
Epoch 8/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0260 - root_mean_squared_error: 0.1613
Epoch 9/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0212 - root_mean_squared_error: 0.1455
Epoch 10/10

2/2 [==============================] - 2s 8ms/step - loss: 0.1869 - root_mean_squared_error: 0.4324
Epoch 2/10
2/2 [==============================] - 0s 5ms/step - loss: 0.1727 - root_mean_squared_error: 0.4155
Epoch 3/10
2/2 [==============================] - 0s 5ms/step - loss: 0.1455 - root_mean_squared_error: 0.3815
Epoch 4/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0502 - root_mean_squared_error: 0.2241
Epoch 5/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0878 - root_mean_squared_error: 0.2963
Epoch 6/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0400 - root_mean_squared_error: 0.2000
Epoch 7/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0558 - root_mean_squared_error: 0.2362
Epoch 8/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0484 - root_mean_squared_error: 0.2199
Epoch 9/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0257 - root_mean_squared_error: 0.1603
Epoch 10/10


2/2 [==============================] - 2s 7ms/step - loss: 0.1798 - root_mean_squared_error: 0.4240
Epoch 2/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0553 - root_mean_squared_error: 0.2351
Epoch 3/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0506 - root_mean_squared_error: 0.2250
Epoch 4/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0290 - root_mean_squared_error: 0.1702
Epoch 5/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0271 - root_mean_squared_error: 0.1648
Epoch 6/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0190 - root_mean_squared_error: 0.1377
Epoch 7/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0228 - root_mean_squared_error: 0.1509
Epoch 8/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0224 - root_mean_squared_error: 0.1497
Epoch 9/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0181 - root_mean_squared_error: 0.1346
Epoch 10/10


In [43]:
df_res = pd.DataFrame(finish, columns = ['combination', 'MAPE','Direction'])
df_res

,combination,MAPE,Direction
0,"(Calm,)",1.714,0.529412
1,"(Kind,)",1.752,0.529412
2,"(Happy,)",2.473,0.529412
3,"(Alert,)",0.750,0.647059
4,"(Sure,)",1.143,0.588235
...,...,...,...
58,"(Calm, Kind, Happy, Sure, Vital)",1.860,0.529412
59,"(Calm, Kind, Alert, Sure, Vital)",3.978,0.588235
60,"(Calm, Happy, Alert, Sure, Vital)",1.856,0.647059
61,"(Kind, Happy, Alert, Sure, Vital)",1.507,0.647059


In [47]:
df_res.query(' Direction > 0.7').reset_index(drop=True)

,combination,MAPE,Direction
0,"(Vital,)",2.440,0.705882
1,"(Kind, Vital)",2.667,0.764706
2,"(Happy, Vital)",8.483,0.764706
3,"(Kind, Alert, Vital)",8.467,0.764706
4,"(Kind, Alert, Sure, Vital)",1.030,0.705882


In [60]:
candidate = {
    1:[['Vital']] ,
    2:[['Kind', 'Vital'], ['Happy', 'Vital']],
    3:[['Kind', 'Alert', 'Vital']],
    4:[['Kind', 'Alert', 'Sure', 'Vital']]
}
for features in candidate:
    print(features)
    for comb in candidate[features]:
        print(comb)

1
['Vital']
2
['Kind', 'Vital']
['Happy', 'Vital']
3
['Kind', 'Alert', 'Vital']
4
['Kind', 'Alert', 'Sure', 'Vital']


In [61]:
feature_num = []
combination = []
MAPE = []
correction = []

for features in candidate:
    for comb in candidate[features]:
        for i in range(100):
            # Data
            NUM_FEATURE = features+1
            k = ['Close']+comb
            tmp = temp2[k]
            X2, y2 = df_to_X_y2(tmp, WINDOW_SIZE)
            X_train2, y_train2 = X2[:threshold-3], y2[:threshold-3]
            X_test2, y_test2 = X2[threshold:], y2[threshold:]
            
            # Modal
            model_k = Sequential()
            model_k.add(InputLayer((WINDOW_SIZE, NUM_FEATURE)))
            model_k.add(LSTM(64))
            model_k.add(Dense(8, 'relu'))
            model_k.add(Dense(1, 'linear'))
            model_k.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.01), metrics=[RootMeanSquaredError()])
            model_k.fit(X_train2, y_train2, epochs=10)
            
            # Test result
            test_predictions2 = model_k.predict(X_test2).flatten()
            pred = mms_close.inverse_transform(test_predictions2.reshape(-1, 1))
            pred = pred.reshape(-1,)
            act = mms_close.inverse_transform(y_test2.reshape(-1, 1))
            act = act.reshape(-1,)
            test_results2 = pd.DataFrame(data={
                'Test Predictions':pred, 
                'Actuals':act})

            # Part 2: past three days' DJIA data + Opinion Finder data
            feature_num.append(features)
            combination.append(comb)
            MAPE.append(mape(test_results2['Actuals'], test_results2['Test Predictions']).round(3))
            correction.append(direction(test_results2['Actuals'], test_results2['Test Predictions']))
            del model_k

Epoch 1/10
2/2 [==============================] - 2s 6ms/step - loss: 0.2681 - root_mean_squared_error: 0.5178
Epoch 2/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0891 - root_mean_squared_error: 0.2985
Epoch 3/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0523 - root_mean_squared_error: 0.2286
Epoch 4/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0559 - root_mean_squared_error: 0.2365
Epoch 5/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0343 - root_mean_squared_error: 0.1851
Epoch 6/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0411 - root_mean_squared_error: 0.2026
Epoch 7/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0329 - root_mean_squared_error: 0.1814
Epoch 8/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0231 - root_mean_squared_error: 0.1519
Epoch 9/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0276 - root_mean_squared_error: 0.1661
E

2/2 [==============================] - 2s 19ms/step - loss: 0.3300 - root_mean_squared_error: 0.5745
Epoch 2/10
2/2 [==============================] - 0s 5ms/step - loss: 0.2341 - root_mean_squared_error: 0.4839
Epoch 3/10
2/2 [==============================] - 0s 5ms/step - loss: 0.1103 - root_mean_squared_error: 0.3322
Epoch 4/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0831 - root_mean_squared_error: 0.2882
Epoch 5/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0705 - root_mean_squared_error: 0.2655
Epoch 6/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0415 - root_mean_squared_error: 0.2037
Epoch 7/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0505 - root_mean_squared_error: 0.2248
Epoch 8/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0437 - root_mean_squared_error: 0.2090
Epoch 9/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0240 - root_mean_squared_error: 0.1548
Epoch 10/10

2/2 [==============================] - 2s 19ms/step - loss: 0.2495 - root_mean_squared_error: 0.4995
Epoch 2/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0670 - root_mean_squared_error: 0.2588
Epoch 3/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0680 - root_mean_squared_error: 0.2607
Epoch 4/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0420 - root_mean_squared_error: 0.2050
Epoch 5/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0390 - root_mean_squared_error: 0.1975
Epoch 6/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0289 - root_mean_squared_error: 0.1701
Epoch 7/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0200 - root_mean_squared_error: 0.1415
Epoch 8/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0278 - root_mean_squared_error: 0.1667
Epoch 9/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0190 - root_mean_squared_error: 0.1378
Epoch 10/10

2/2 [==============================] - 2s 13ms/step - loss: 0.2293 - root_mean_squared_error: 0.4789
Epoch 2/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0867 - root_mean_squared_error: 0.2944
Epoch 3/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0722 - root_mean_squared_error: 0.2686
Epoch 4/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0435 - root_mean_squared_error: 0.2085
Epoch 5/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0546 - root_mean_squared_error: 0.2337
Epoch 6/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0437 - root_mean_squared_error: 0.2091
Epoch 7/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0257 - root_mean_squared_error: 0.1602
Epoch 8/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0316 - root_mean_squared_error: 0.1778
Epoch 9/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0239 - root_mean_squared_error: 0.1546
Epoch 10/10

2/2 [==============================] - 3s 5ms/step - loss: 0.1861 - root_mean_squared_error: 0.4314
Epoch 2/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0695 - root_mean_squared_error: 0.2637
Epoch 3/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0422 - root_mean_squared_error: 0.2055
Epoch 4/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0483 - root_mean_squared_error: 0.2198
Epoch 5/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0503 - root_mean_squared_error: 0.2244
Epoch 6/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0232 - root_mean_squared_error: 0.1523
Epoch 7/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0284 - root_mean_squared_error: 0.1684
Epoch 8/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0327 - root_mean_squared_error: 0.1808
Epoch 9/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0202 - root_mean_squared_error: 0.1422
Epoch 10/10


...

2/2 [==============================] - 3s 6ms/step - loss: 0.3176 - root_mean_squared_error: 0.5636
Epoch 2/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0991 - root_mean_squared_error: 0.3149
Epoch 3/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0965 - root_mean_squared_error: 0.3106
Epoch 4/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0538 - root_mean_squared_error: 0.2319
Epoch 5/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0564 - root_mean_squared_error: 0.2374
Epoch 6/10
2/2 [==============================] - 0s 8ms/step - loss: 0.0650 - root_mean_squared_error: 0.2550
Epoch 7/10
2/2 [==============================] - 0s 8ms/step - loss: 0.0433 - root_mean_squared_error: 0.2081
Epoch 8/10
2/2 [==============================] - 0s 8ms/step - loss: 0.0312 - root_mean_squared_error: 0.1768
Epoch 9/10
2/2 [==============================] - 0s 7ms/step - loss: 0.0383 - root_mean_squared_error: 0.1958
Epoch 10/10


2/2 [==============================] - 3s 6ms/step - loss: 0.1922 - root_mean_squared_error: 0.4384
Epoch 2/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0646 - root_mean_squared_error: 0.2542
Epoch 3/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0445 - root_mean_squared_error: 0.2110
Epoch 4/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0501 - root_mean_squared_error: 0.2239
Epoch 5/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0272 - root_mean_squared_error: 0.1649
Epoch 6/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0392 - root_mean_squared_error: 0.1980
Epoch 7/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0241 - root_mean_squared_error: 0.1553
Epoch 8/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0334 - root_mean_squared_error: 0.1828
Epoch 9/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0203 - root_mean_squared_error: 0.1425
Epoch 10/10


2/2 [==============================] - 3s 5ms/step - loss: 0.3306 - root_mean_squared_error: 0.5749
Epoch 2/10
2/2 [==============================] - 0s 4ms/step - loss: 0.1730 - root_mean_squared_error: 0.4159
Epoch 3/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0692 - root_mean_squared_error: 0.2631
Epoch 4/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0934 - root_mean_squared_error: 0.3055
Epoch 5/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0511 - root_mean_squared_error: 0.2261
Epoch 6/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0615 - root_mean_squared_error: 0.2480
Epoch 7/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0591 - root_mean_squared_error: 0.2431
Epoch 8/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0372 - root_mean_squared_error: 0.1930
Epoch 9/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0307 - root_mean_squared_error: 0.1752
Epoch 10/10


2/2 [==============================] - 2s 5ms/step - loss: 0.1634 - root_mean_squared_error: 0.4042
Epoch 2/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0684 - root_mean_squared_error: 0.2615
Epoch 3/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0378 - root_mean_squared_error: 0.1945
Epoch 4/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0226 - root_mean_squared_error: 0.1504
Epoch 5/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0215 - root_mean_squared_error: 0.1467
Epoch 6/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0251 - root_mean_squared_error: 0.1585
Epoch 7/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0174 - root_mean_squared_error: 0.1320
Epoch 8/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0219 - root_mean_squared_error: 0.1478
Epoch 9/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0147 - root_mean_squared_error: 0.1214
Epoch 10/10


2/2 [==============================] - 3s 7ms/step - loss: 0.1271 - root_mean_squared_error: 0.3565
Epoch 2/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0879 - root_mean_squared_error: 0.2964
Epoch 3/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0362 - root_mean_squared_error: 0.1902
Epoch 4/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0568 - root_mean_squared_error: 0.2382
Epoch 5/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0269 - root_mean_squared_error: 0.1640
Epoch 6/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0444 - root_mean_squared_error: 0.2107
Epoch 7/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0220 - root_mean_squared_error: 0.1483
Epoch 8/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0318 - root_mean_squared_error: 0.1784
Epoch 9/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0188 - root_mean_squared_error: 0.1373
Epoch 10/10


In [64]:
df_res = pd.DataFrame(
    {'feature_num':feature_num,
     'combination':combination,
     'MAPE': MAPE,
     'Direction': correction})

In [114]:
df_V = df_res.iloc[:100]
df_KV = df_res.iloc[100:200]
df_HV = df_res.iloc[200:300]
df_KAV = df_res.iloc[300:400]
df_KASV = df_res.iloc[400:]

In [120]:
print('Vital:')
print(df_V.describe())
print('-'*10)

print('Kind, Vital:')
print(df_KV.describe())
print('-'*10)

print('Happy, Vital:')
print(df_HV.describe())
print('-'*10)

print('Kind, Alert, Vital:')
print(df_KAV.describe())
print('-'*10)

print('Kind, Alert, Sure, Vital:')
print(df_KASV.describe())

Vital:
       feature_num        MAPE   Direction
count        100.0  100.000000  100.000000
mean           1.0    1.774090    0.626471
std            0.0    1.268578    0.088631
min            1.0    0.712000    0.352941
25%            1.0    0.983500    0.588235
50%            1.0    1.418000    0.647059
75%            1.0    2.222750    0.705882
max            1.0    8.491000    0.764706
----------
Kind, Vital:
       feature_num        MAPE   Direction
count        100.0  100.000000  100.000000
mean           2.0    1.463350    0.647647
std            0.0    0.681542    0.071677
min            2.0    0.772000    0.411765
25%            2.0    0.925000    0.588235
50%            2.0    1.201000    0.647059
75%            2.0    1.946000    0.705882
max            2.0    4.780000    0.764706
----------
Happy, Vital:
       feature_num        MAPE   Direction
count        100.0  100.000000  100.000000
mean           2.0    1.695590    0.591765
std            0.0    1.236294    0.05406

In [106]:
# Summary
combination = []
MAPE = []
Correction = []

j = 0
for i in range(100, 501, 100):
    tmp = df_res.iloc[j:i]
    tmp = tmp.groupby(by='feature_num').agg({'combination':'first',
                                              'MAPE':np.mean,
                                              'Direction':np.mean}).reset_index(drop=True)
    combination.append(tmp['combination'][0])
    MAPE.append(tmp['MAPE'][0])
    Correction.append(tmp['Direction'][0])
    j = i

df_final = pd.DataFrame({'combination':combination,
                         'MAPE':MAPE,
                         'Correction':Correction})

In [107]:
df_final

,combination,MAPE,Correction
0,[Vital],1.77409,0.626471
1,"[Kind, Vital]",1.46335,0.647647
2,"[Happy, Vital]",1.69559,0.591765
3,"[Kind, Alert, Vital]",1.54511,0.617059
4,"[Kind, Alert, Sure, Vital]",1.60931,0.617647


reference: 

https://colab.research.google.com/drive/1b3CUJuDOmPmNdZFH3LQDmt5F0K3FZhqD?usp=sharing#scrollTo=FmwshpETs-jE